<a href="https://colab.research.google.com/github/mfahadakbar/Feature_selection_with_Corr/blob/master/FE_with_Corr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#keeping in view corelation with the taget variable . We will call this fuunction whenever we need to reduce our dimentions
  def c_f(input_data,threshold=Corr_TH,tar=target):

    global data1
    # make an corelation db with abs corelation db
    corr_matrix = abs(data1.corr())

    # for every diagonal value, make it Nan
    corr_matrix.values[tuple([np.arange(corr_matrix.shape[0])]*2)] = np.NaN

    # Now Calculate the average correlation of every feature with other, and get a pandas data frame
    avg_cor = pd.DataFrame(corr_matrix.mean())
    avg_cor['feature']= avg_cor.index
    avg_cor.reset_index(drop=True, inplace=True)
    avg_cor.columns =  ['avg_cor','features']
    
    # Calculate the corelation with the target
    targ_cor = pd.DataFrame(corr_matrix[tar].dropna())
    targ_cor['feature']= targ_cor.index
    targ_cor.reset_index(drop=True, inplace=True)
    targ_cor.columns =  ['target_cor','features']

    # Now, add a column for variable name and drop index
    corr_matrix['column'] = corr_matrix.index
    corr_matrix.reset_index(drop=True,inplace=True)

    # now we need to melt it , so that we can corelation pair wise , with two columns 
    cols =corr_matrix.column
    melt = corr_matrix.melt(id_vars= ['column'],value_vars=cols).sort_values(by='value',ascending=False).dropna()

    # now bring in the avg corelation for first of the pair
    merge = pd.merge(melt,avg_cor,left_on='column',right_on='features').drop('features',axis=1)

    # now bring in the avg corelation for second of the pair
    merge = pd.merge(merge,avg_cor,left_on='variable',right_on='features').drop('features',axis=1)
    
    # now bring in the target corelation for first of the pair
    merge = pd.merge(merge,targ_cor,left_on='column',right_on='features').drop('features',axis=1)

    # now bring in the avg corelation for second of the pair
    merge = pd.merge(merge,targ_cor,left_on='variable',right_on='features').drop('features',axis=1)

    # sort and save
    merge = merge.sort_values(by='value',ascending=False)

    # we need to now eleminate all the pairs that are actually duplicate e.g cor(x,y) = cor(y,x) , they are the same , we need to find these and drop them
    merge['all_columns'] = merge['column'] + merge['variable']

    # this puts all the coresponding pairs of features togather , so that we can only take one, since they are just the duplicates
    merge['all_columns'] = [sorted(i) for i in merge['all_columns'] ]

    # now sort by new column
    merge = merge.sort_values(by='all_columns')

    # take every second colums
    merge = merge.iloc[::2, :]

    # make a rinking column to eliminate features
    merge['rank_x'] = round((merge['avg_cor_y']- merge['avg_cor_x']) + (merge['target_cor_x'] - merge['target_cor_y']),6) # round it to 6 digits

    ## Now there will be rows where the rank will be exactly zero, these is where the value (corelartion between features) is exactly one ( like price and price^2)
    ## so in that case , we can simply pick one of the variable
    # but since , features can be in either column, we will drop one column (say 'column') , only if the feature is not in the second column (in variable column)
    # both equations below will return the list of columns to drop from here 
    # this is how it goes

    ## For the portion where corelation is exactly one !
    one = merge[merge['rank_x']==0]

    #[i for i in pd.unique(small['column']) if i not in pd.unique(small['variable'])]
    to_drop =(list(set(one['column'])-set(one['variable'])))

    ## now we are to treat where rank is not Zero and Value (corelation) is greater than a specific threshold
    non_zero = merge[(merge['rank_x']!= 0.0) & (merge['value'] >= Corr_TH)]

    # pick the column to delete
    non_zero_list = list(np.where(non_zero['rank_x'] < 0 , non_zero['column'], non_zero['variable']))

    # add two list
    to_drop = to_drop + non_zero_list

    #make sure that target column is not a part of the list
    try:
      to_drop.remove(tar)
    except:
      to_drop

    #Final Step, drop the columns from data
    data1.drop(to_drop,axis=1,inplace=True)

NameError: ignored

In [0]:
import numpy as np
import pandas as pd
data1 = pd.read_csv('/content/drive/My Drive/Steve_Assignments/OJ_copy_transformed.csv')

In [0]:
Corr_TH = 0.45
target = 'Purchase'
c_f(data1)

In [0]:
data.dtypes

Unnamed: 0                                     int64
LoyalCH / WeekofPurchase / WeekofPurchase    float64
LoyalCH / WeekofPurchase + PctDiscCH^2       float64
LoyalCH / WeekofPurchase * STORE_4           float64
WeekofPurchase                                 int64
StoreID_1                                      int64
STORE_2                                        int64
Purchase                                       int64
dtype: object